In [1]:
import tensorflow as tf
import numpy as np
import parser

# Dynamic GPU memory growth
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [27]:
def singular_values(X):
    s, u, v = tf.svd(X)
    return s

In [28]:
def matrix_symmetric(x):
    return (x + tf.transpose(x, [0,2,1])) / 2

In [29]:
def get_eigen_K(x, square=False):
    """
    Get K = 1 / (sigma_i - sigma_j) for i != j, 0 otherwise

    Parameters
    ----------
    x : tf.Tensor with shape as [..., dim,]

    Returns
    -------

    """
    if square:
        x = tf.square(x)
    res = tf.expand_dims(x, 1) - tf.expand_dims(x, 2)
    res += tf.eye(tf.shape(res)[1])
    res = 1 / res
    res -= tf.eye(tf.shape(res)[1])

    # Keep the results clean
    res = tf.where(tf.is_nan(res), tf.zeros_like(res), res)
    res = tf.where(tf.is_inf(res), tf.zeros_like(res), res)
    return res


In [30]:
@tf.RegisterGradient('Svd')
def gradient_svd(op, grad_s, grad_u, grad_v):
    """
    Define the gradient for SVD
    References
        Ionescu, C., et al, Matrix Backpropagation for Deep Networks with Structured Layers
        
    Parameters
    ----------
    op
    grad_s
    grad_u
    grad_v

    Returns
    -------
    """
    s, u, v = op.outputs
    v_t = tf.transpose(v, [0,2,1])

    with tf.name_scope('K'):
        K = get_eigen_K(s, True)
    inner = matrix_symmetric(K * tf.matmul(v_t, grad_v))

    # Create the shape accordingly.
    u_shape = u.get_shape()[1].value
    v_shape = v.get_shape()[1].value

    # Recover the complete S matrices and its gradient
    eye_mat = tf.eye(v_shape, u_shape)
    realS = tf.matmul(tf.reshape(tf.matrix_diag(s), [-1, v_shape]), eye_mat)
    realS = tf.transpose(tf.reshape(realS, [-1, v_shape, u_shape]), [0, 2, 1])

    real_grad_S = tf.matmul(tf.reshape(tf.matrix_diag(grad_s), [-1, v_shape]), eye_mat)
    real_grad_S = tf.transpose(tf.reshape(real_grad_S, [-1, v_shape, u_shape]), [0, 2, 1])

    dxdz = tf.matmul(u, tf.matmul(2 * tf.matmul(realS, inner) + real_grad_S, v_t))
    return dxdz

KeyError: "Registering two gradient with name 'Svd' !(Previous registration was in <module> /usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16)"

In [38]:
def find_reprs(X_test, X_train, lambda_val = 1, learning_rate=1):
    X = tf.placeholder("float", X_test.shape)

    D = tf.placeholder("float", (X_train.shape[1], X_train.shape[0]))
    R = tf.placeholder("float", (X_train.shape[0], X_test.shape[0]))

    R_weights = tf.Variable(tf.random_normal([X_train.shape[0], X_test.shape[0]]))

    minimization_term = tf.norm(X - tf.transpose(tf.matmul(D, R_weights)), ord='fro', axis=(0,1))
    regularization_term = lambda_val * tf.reduce_sum(singular_values(R_weights))
    loss = minimization_term + regularization_term
    loss /= X_test.shape[0]
    optimizer = tf.train.AdadeltaOptimizer(learning_rate)

    R_new = optimizer.minimize(loss, var_list=[R_weights])

    init = tf.global_variables_initializer()
    sess.run(init)

    prev_error = np.inf
    max_epochs = 1000

    # Train model
    R_train = None
    for i in range(max_epochs):

        # Update R
        _, l, R_train = sess.run([R_new, loss, R_weights],feed_dict={X: X_test, D: X_train.T})
        if l > prev_error:
            break
        prev_error = l	
        #print("Representation finding loss (MSE) : %.4f" % (l))

    return R_train.T

In [39]:
def main():
    parser.init()

    average = 0.0
    for i in range(1,6):
        (X_train, Y_train), (X_test, Y_test) = parser.get_data_split(i)
        X_train = X_train[:10000]
        Y_train = Y_train[:10000]
        features = find_reprs(X_test.astype("float"), X_train.astype("float"), 0.01, 200)
        indices = np.argmax(features, axis=1)
        Y_predict = []
        for j in indices:
            Y_predict.append(Y_train[j])
        Y_predict = np.array(Y_predict)
        score = np.sum(1*(np.argmax(Y_predict, axis=1) == np.argmax(Y_test, axis=1)))
        score /= float(1000)
        print score
        average += score
    print average / float(5)


In [40]:
main()

ValueError: Dimension must be 2 but is 3 for 'gradients_5/Svd_8_grad/transpose' (op: 'Transpose') with input shapes: [20000,10000], [3].